# Peer assignment - Segmenting and clustering neighborhoods in Toronto

### Importing library

In [2]:
import pandas as pd

### Scraping from wikipedia

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = pd.DataFrame(dfs[0][['Postal Code','Borough','Neighbourhood']])

### Remove not assigned in borough

In [4]:
df_filtered = df[df['Borough'] != 'Not assigned'] 
df_filtered

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Merge Neightborhood with the same postal code

In [5]:
df_filtered = df_filtered.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_filtered

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### Assigned Neightbourhood to Borough if Neighbourhood is not assigned

In [6]:
df_new = pd.DataFrame(df_filtered)
df_new.loc[df_new['Neighbourhood'] == 'Not assigned',['Neighbourhood'] ] = df_new['Borough']
df_new

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### Shaping

In [7]:
df_new.shape

(103, 3)

### Import csv that contains longitude and latitude

In [8]:
df_coordinate = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinate

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
df_with_coor = df_new
df_with_coor['Latitude'] = df_coordinate.loc[df_coordinate['Postal Code'] == df_with_coor['Postal Code'], ['Latitude']] = df_coordinate['Latitude']
df_with_coor['Longitude'] = df_coordinate.loc[df_coordinate['Postal Code'] == df_with_coor['Postal Code'], ['Longitude']] = df_coordinate['Longitude']
df_with_coor

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Analyzing and clustring

### Import and download folium

In [18]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [22]:
import folium
from geopy.geocoders import Nominatim

### Getting longitude and latitude for toronto

In [23]:
address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The longitude{} and The latitude{}'.format(longitude,latitude))

The longitude-79.3839347 and The latitude43.6534817


### Create Map of Toronto

In [26]:
toronto_map = folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df_with_coor['Latitude'],df_with_coor['Longitude'],df_with_coor['Borough'],df_with_coor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(labelmparse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        full_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
toronto_map